# **Glove**

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-04-11 18:22:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-11 18:22:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-11 18:22:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
def loadGlove(path):
  file = open(path, 'r')
  model = {}
  for l in file:
    line = l.split()
    word = line[0]
    value = np.array([float(val) for val in line[1:]])
    model[word] = value 
  return model

glove = loadGlove('/content/glove.6B.50d.txt')

In [ ]:
glove['python']

array([ 0.5897  , -0.55043 , -1.0106  ,  0.41226 ,  0.57348 ,  0.23464 ,
       -0.35773 , -1.78    ,  0.10745 ,  0.74913 ,  0.45013 ,  1.0351  ,
        0.48348 ,  0.47954 ,  0.51908 , -0.15053 ,  0.32474 ,  1.0789  ,
       -0.90894 ,  0.42943 , -0.56388 ,  0.69961 ,  0.13501 ,  0.16557 ,
       -0.063592,  0.35435 ,  0.42819 ,  0.1536  , -0.47018 , -1.0935  ,
        1.361   , -0.80821 , -0.674   ,  1.2606  ,  0.29554 ,  1.0835  ,
        0.2444  , -1.1877  , -0.60203 , -0.068315,  0.66256 ,  0.45336 ,
       -1.0178  ,  0.68267 , -0.20788 , -0.73393 ,  1.2597  ,  0.15425 ,
       -0.93256 , -0.15025 ])

In [ ]:
cosine_similarity(glove['cat'].reshape(1, -1), glove['dog'].reshape(1, -1))

array([[0.92180053]])

In [ ]:
glove['python'].reshape(1, -1)

array([[ 0.5897  , -0.55043 , -1.0106  ,  0.41226 ,  0.57348 ,  0.23464 ,
        -0.35773 , -1.78    ,  0.10745 ,  0.74913 ,  0.45013 ,  1.0351  ,
         0.48348 ,  0.47954 ,  0.51908 , -0.15053 ,  0.32474 ,  1.0789  ,
        -0.90894 ,  0.42943 , -0.56388 ,  0.69961 ,  0.13501 ,  0.16557 ,
        -0.063592,  0.35435 ,  0.42819 ,  0.1536  , -0.47018 , -1.0935  ,
         1.361   , -0.80821 , -0.674   ,  1.2606  ,  0.29554 ,  1.0835  ,
         0.2444  , -1.1877  , -0.60203 , -0.068315,  0.66256 ,  0.45336 ,
        -1.0178  ,  0.68267 , -0.20788 , -0.73393 ,  1.2597  ,  0.15425 ,
        -0.93256 , -0.15025 ]])

In [ ]:
cosine_similarity(glove['cat'].reshape(1, -1), glove['piano'].reshape(1, -1))

array([[0.19825255]])

In [ ]:
predicted_king_embedding = glove['queen'] - glove['woman'] + glove['man']
cosine_similarity(predicted_king_embedding.reshape(1, -1), glove['king'].reshape(1, -1))

array([[0.85888392]])

In [ ]:
predicted_emphasis_embedding_1 = glove['softer'] - glove['soft']
predicted_emphasis_embedding_2 = glove['darker'] - glove['dark']
cosine_similarity(predicted_emphasis_embedding_1.reshape(1, -1), predicted_emphasis_embedding_2.reshape(1, -1))

array([[0.52087747]])

# **CBOW**

In [ ]:
import torch 
import torch.nn as nn
import numpy as np

In [ ]:
text = """How that personage haunted my dreams, I need scarcely tell you. On
stormy nights, when the wind shook the four corners of the house and
the surf roared along the cove and up the cliffs, I would see him in a
thousand forms, and with a thousand diabolical expressions. Now the leg
would be cut off at the knee, now at the hip, now he was a monstrous
kind of a creature who had never had but the one leg, and that in the
middle of his body. To see him leap and run and pursue me over hedge and
ditch was the worst of nightmares. And altogether I paid pretty dear for
my monthly fourpenny piece, in the shape of these abominable fancies"""

In [ ]:
text = text.replace(',', '').replace('.', '').lower().split()

In [ ]:
corpus = set(text)
corpus_length = len(corpus)

word_dict = {}
inverse_word_dict = {}

for i, word in enumerate(corpus):
  word_dict[word] = i
  inverse_word_dict[i] = word 


data = []

for i in range(2, len(text) - 2):
  sentence = [text[i-2], text[i-1], text[i+1], text[i+2]]
  target = text[i]
  data.append((sentence, target))

print(data[3])

(['haunted', 'my', 'i', 'need'], 'dreams')


In [ ]:
set(text)

{'a',
 'abominable',
 'along',
 'altogether',
 'and',
 'at',
 'be',
 'body',
 'but',
 'cliffs',
 'corners',
 'cove',
 'creature',
 'cut',
 'dear',
 'diabolical',
 'ditch',
 'dreams',
 'expressions',
 'fancies',
 'for',
 'forms',
 'four',
 'fourpenny',
 'had',
 'haunted',
 'he',
 'hedge',
 'him',
 'hip',
 'his',
 'house',
 'how',
 'i',
 'in',
 'kind',
 'knee',
 'leap',
 'leg',
 'me',
 'middle',
 'monstrous',
 'monthly',
 'my',
 'need',
 'never',
 'nightmares',
 'nights',
 'now',
 'of',
 'off',
 'on',
 'one',
 'over',
 'paid',
 'personage',
 'piece',
 'pretty',
 'pursue',
 'roared',
 'run',
 'scarcely',
 'see',
 'shape',
 'shook',
 'stormy',
 'surf',
 'tell',
 'that',
 'the',
 'these',
 'thousand',
 'to',
 'up',
 'was',
 'when',
 'who',
 'wind',
 'with',
 'worst',
 'would',
 'you'}

In [ ]:
embedding_length = 20

class CBOW(torch.nn.Module):

  def __init__(self, corpus_length, embedding_dim):
    super(CBOW, self).__init__()

    self.embeddings = nn.Embedding(corpus_length, embedding_dim)

    self.linear1 = nn.Linear(embedding_dim, 64)
    self.linear2 = nn.Linear(64, corpus_length)

    self.activation_function1 = nn.ReLU()
    self.activation_function2 = nn.LogSoftmax( dim = -1)

  def forward(self, inputs):
    embeds = sum(self.embeddings(inputs)).view(1, -1)
    out = self.linear1(embeds)
    out = self.activation_function1(out)
    out = self.linear2(out)
    out = self.activation_function2(out)
    return out 

  def get_word_embeddings(self, word):
    word = torch.LongTensor([word_dict[word]])
    return self.embeddings(word).view(1, -1)

In [ ]:
model = CBOW(corpus_length, embedding_length)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def make_sentence_vector(sentence, word_dict):
  idxs = [word_dict[w] for w in sentence]
  return torch.tensor(idxs, dtype=torch.long)

print(make_sentence_vector(['stormy', 'nights', 'when', 'the'], word_dict))

tensor([64, 58, 50, 20])


In [ ]:
for epoch in range(100):
  epoch_loss = 0
  for sentence, target in data:
    model.zero_grad()
    sentence_vector = make_sentence_vector(sentence, word_dict)
    log_probs = model(sentence_vector)
    loss = loss_function(log_probs, torch.tensor([word_dict[target]], dtype=torch.long))
    loss.backward()
    optimizer.step()
    epoch_loss = epoch_loss + loss.data
  print('Epoch: '+str(epoch)+', Loss: ' + str(epoch_loss.item()))

Epoch: 0, Loss: 529.4207153320312
Epoch: 1, Loss: 474.18426513671875
Epoch: 2, Loss: 432.6933898925781
Epoch: 3, Loss: 395.2223205566406
Epoch: 4, Loss: 358.1815490722656
Epoch: 5, Loss: 320.8206787109375
Epoch: 6, Loss: 282.588134765625
Epoch: 7, Loss: 243.8424835205078
Epoch: 8, Loss: 205.9978485107422
Epoch: 9, Loss: 170.1129150390625
Epoch: 10, Loss: 137.52908325195312
Epoch: 11, Loss: 109.46037292480469
Epoch: 12, Loss: 86.3451156616211
Epoch: 13, Loss: 68.09696197509766
Epoch: 14, Loss: 54.076900482177734
Epoch: 15, Loss: 43.386436462402344
Epoch: 16, Loss: 35.392494201660156
Epoch: 17, Loss: 29.198354721069336
Epoch: 18, Loss: 24.522384643554688
Epoch: 19, Loss: 20.869436264038086
Epoch: 20, Loss: 18.005041122436523
Epoch: 21, Loss: 15.754572868347168
Epoch: 22, Loss: 13.970394134521484
Epoch: 23, Loss: 12.510107040405273
Epoch: 24, Loss: 11.320457458496094
Epoch: 25, Loss: 10.314642906188965
Epoch: 26, Loss: 9.460220336914062
Epoch: 27, Loss: 8.731156349182129
Epoch: 28, Loss: 

In [ ]:
def get_predicted_result(input, inverse_word_dict):
  index = np.argmax(input)
  return inverse_word_dict[index]

In [ ]:
def predict_sentence(sentence):
  sentence_split = sentence.replace('.', '').lower().split()
  sentence_vector = make_sentence_vector(sentence_split, word_dict)
  prediction_array = model(sentence_vector).data.numpy()
  print('Preceding Words: {}\n'.format(sentence_split[:2]))
  print('Predicted Word: {}\n'.format(get_predicted_result(prediction_array[0], inverse_word_dict)))
  print('Following Words: {}\n'.format(sentence_split[2:]))

In [ ]:
predict_sentence('to see leap and')

Preceding Words: ['to', 'see']

Predicted Word: him

Following Words: ['leap', 'and']



In [ ]:
print(model.get_word_embeddings('leap'))

tensor([[-0.1024,  1.0129, -2.1892, -0.3026,  0.3400, -1.9102,  0.0484, -0.8698,
          0.3539, -0.6314, -0.2415,  0.8329,  0.0054, -0.1078, -0.5404,  0.5172,
          0.7934,  0.5753, -0.2397, -1.9517]], grad_fn=<ViewBackward>)


# **Tokenization**

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [ ]:
text = 'This is a single sentence.'
tokens = word_tokenize(text)
print(tokens)

['This', 'is', 'a', 'single', 'sentence', '.']


In [ ]:
no_punctuation = [word.lower() for word in tokens if word.isalpha()]
print(no_punctuation)

['this', 'is', 'a', 'single', 'sentence']


In [ ]:
text = "This is the first sentence. This is the second sentence. A document contains many sentences."
print(sent_tokenize(text))

['This is the first sentence.', 'This is the second sentence.', 'A document contains many sentences.']


In [ ]:
print([word_tokenize(sentence) for sentence in sent_tokenize(text)])

[['This', 'is', 'the', 'first', 'sentence', '.'], ['This', 'is', 'the', 'second', 'sentence', '.'], ['A', 'document', 'contains', 'many', 'sentences', '.']]


In [ ]:
nltk.download("stopwords") 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')
print(stop_words[:20])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [ ]:
text = 'This is a single sentence.'
tokens = [token for token in word_tokenize(text) if token not in stop_words]
print(tokens)

['This', 'single', 'sentence', '.']


# **POS Tagging and Chunking**

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [ ]:
sentence = 'The big dog is sleeping on the bed'
tokens = nltk.word_tokenize(sentence)
nltk.pos_tag(tokens)

[('The', 'DT'),
 ('big', 'JJ'),
 ('dog', 'NN'),
 ('is', 'VBZ'),
 ('sleeping', 'VBG'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('bed', 'NN')]

In [ ]:
nltk.help.upenn_tagset("VBG")

VBG: verb, present participle or gerund
    telegraphing stirring focusing angering judging stalling lactating
    hankerin' alleging veering capping approaching traveling besieging
    encrypting interrupting erasing wincing ...


In [ ]:
tagged = nltk.pos_tag(tokens)

In [ ]:
tagged

[('The', 'DT'),
 ('big', 'JJ'),
 ('dog', 'NN'),
 ('is', 'VBZ'),
 ('sleeping', 'VBG'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('bed', 'NN')]

In [ ]:
expression = ('NP: {<DT>?<JJ>*<NN>}')
REchunkParser = nltk.RegexpParser(expression)
tree = REchunkParser.parse(tagged)

In [ ]:
print(tree)

(S
  (NP The/DT big/JJ dog/NN)
  is/VBZ
  sleeping/VBG
  on/IN
  (NP the/DT bed/NN))


# **TF-IDF**

In [ ]:
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
emma = nltk.corpus.gutenberg.sents('austen-emma.txt')

In [ ]:
emma_sentences = []
emma_word_set = []

for sentence in emma:
  emma_sentences.append([word.lower() for word in sentence if word.isalpha()])
  for word in sentence:
    if word.isalpha():
      emma_word_set.append(word.lower())

emma_word_set = set(emma_word_set)

In [ ]:
def TermFreq(document, word):
  doc_length = len(document)
  occurances = len([w for w in document if w == word])
  return occurances / doc_length


TermFreq(emma_sentences[5], 'ago')

0.024390243902439025

In [ ]:
def build_DF_dict():
  output = {}
  for word in emma_word_set:
    output[word] = 0
    for doc in emma_sentences:
      if word in doc:
        output[word] = output[word] + 1
  return output

In [ ]:
df_dict = build_DF_dict()
df_dict['ago']

32

In [ ]:
def InverseDocumentFrequency(word):
  N = len(emma_sentences)
  try:
    df = df_dict[word] + 1
  except:
    df = 1
  return np.log(N/df)

In [ ]:
InverseDocumentFrequency('ago')

5.459198592104122

In [ ]:
def TFIDF(doc, word):
  tf = TermFreq(doc, word)
  idf = InverseDocumentFrequency(word)
  return tf * idf 


In [ ]:
print('ago - ' + str(TFIDF(emma_sentences[5], 'ago')))
print('indistinct - ' + str(TFIDF(emma_sentences[5], 'indistinct')))


ago - 0.13315118517327126
indistinct - 0.20152582861001603


In [ ]:
def loadGlove(path):
  file = open(path, 'r')
  model = {}
  for l in file:
    line = l.split()
    word = line[0]
    value = np.array([float(val) for val in line[1:]])
    model[word] = value
  return model 
  

In [ ]:
glove = loadGlove('glove.6B.50d.txt')

In [ ]:
embeddings = []

for word in emma_sentences[5]:
  embeddings.append(glove[word])

mean_embedding = np.mean(embeddings, axis=0).reshape(1, -1)
print(mean_embedding)

[[ 3.32575634e-01  3.16596488e-01 -1.80050732e-01 -3.82070951e-01
   4.98493527e-01  5.33804805e-01 -5.46517073e-01  9.12476195e-02
  -1.31538483e-01 -2.71967805e-02  2.99867317e-02  2.64278024e-02
  -2.06519756e-01 -1.54796634e-01  4.28036366e-01 -5.74977317e-02
  -2.65928778e-01  1.60373902e-02 -2.84913561e-01 -2.01252268e-01
  -5.96390732e-02  5.72458220e-01  2.06195927e-01 -1.54312293e-01
   2.52049805e-01 -1.64638200e+00 -3.42686049e-01  1.02592522e-01
   1.42848000e-01 -1.09779902e-01  2.89345488e+00  7.36985634e-02
  -3.73648780e-03 -2.76292784e-01  1.50580049e-01  9.80399951e-02
   2.24408780e-03  2.83664024e-01  3.92979024e-02 -2.98091634e-01
  -1.17309171e-01  2.08815776e-01  6.89953902e-03  2.92777244e-02
   5.54180122e-02 -2.20519707e-01 -2.82007805e-01 -4.34917439e-01
  -9.69051537e-02 -1.67569878e-01]]


In [ ]:
embeddings = []

for word in emma_sentences[5]:
  tfidf = TFIDF(emma_sentences[5], word)
  embeddings.append(glove[word] * tfidf)

tfidf_weighted_embedding = np.mean(embeddings, axis=0).reshape(1, -1)
print(tfidf_weighted_embedding)

[[ 0.03390627  0.04567951 -0.02513047 -0.05553374  0.06523389  0.07031937
  -0.06309126  0.02674499 -0.01073998 -0.00509068  0.00518551  0.00818713
  -0.01610237 -0.01486281  0.04954961 -0.0107796  -0.05029558  0.00039276
  -0.0192399  -0.01344365 -0.01123742  0.08506534  0.02145731 -0.0159164
   0.04411737 -0.17889813 -0.04006272  0.01603446  0.02090289 -0.01344211
   0.28346797  0.00696015  0.00484046 -0.02637939  0.01537125  0.01611019
   0.00316879  0.0324516   0.00829024 -0.04200008 -0.0058922   0.01996137
  -0.00305491 -0.00355021  0.01175475 -0.03423196 -0.02943769 -0.06810232
  -0.00775695 -0.0181068 ]]


In [ ]:
cosine_similarity(mean_embedding, tfidf_weighted_embedding)

array([[0.98653879]])

In [2]:
emma_set = set(["I","am", "a", "man", "and", "you", "was", "a", "man", "though" ])
sentences = [["I","am", "a", "man", "and", "a", "man"], ["you", "was", "a", "man", "though"]]




In [3]:
output = {}

for word in emma_set:
  output[word] = 0
  for doc in sentences:
    if word in doc:
      output[word] = output[word] + 1


print(output)


{'was': 1, 'I': 1, 'man': 2, 'am': 1, 'you': 1, 'though': 1, 'and': 1, 'a': 2}
